# Jupyter tutorial for plotting remote data (netcdf or grib -- doesn't matter) from NCAR RDA server
Author: Dan Chavas, Purdue

Last revision: 2019-01-09

This is based off of Ryan Abernathey's [tutorial](https://rabernat.github.io/research_computing/intro-to-basemap.html)  on using Basemap in python -- thanks Ryan!

## Read data directly from a remote source (i.e. NO need to download!) and make a basic plot

### Import relevant packages

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
from netCDF4 import Dataset,num2date,date2num
import subprocess
import os
import shutil
import tempfile
import gzip
import datetime
import argparse
%matplotlib inline
    #above line is specifically for jupyter notebook interface; comment out when running .py script (rather than .ipynb) 
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

import netCDF4
from datetime import datetime, timedelta
# from windspharm.standard import VectorWind
# from windspharm.tools import prep_data, recover_data, order_latdim

### Read data directly from NCAR RDA (no downloading!)

In [ ]:
# set up the URL to access the data server.
# This example is for ERA-Interim Reanalysis data
# See the NCAR RDA for available options

## USER INPUT ########################################
yyyymm_in='201802'
ddhh_in='0306'
date_in='%s%s'%(yyyymm_in,ddhh_in)  # this needs to be a recent data since only recent models are available for download
var_in = 'Temperature_isobaric'  #see output below for other variable options 
i_plev = -1   #in ERA-I, pressure levels go TOA --> surface; thus last entry = level closest to the surface
url='https://rda.ucar.edu/thredds/dodsC/files/e/ds627.0/ei.oper.an.pl/%s/ei.oper.an.pl.regn128sc.%s'%(yyyymm_in,date_in)
############################################################

# Extract the desired data
# The indexing into the data set used by netCDF4 is standard python indexing
file = netCDF4.Dataset(url)
print('Contents of file:',file)    # this prints the contents of the file if you want to see what's inside
lat  = file.variables['lat'][:]
lon  = file.variables['lon'][:]
plev  = file.variables['isobaric'][:]  #[hPa]
plev_in = plev[i_plev] #[hPa]
data = file.variables[var_in][0,i_plev,:,:]  #dims = [time, plev, lat, lon]; "0" = first entry in dimension
file.close()

print('Date = %s'%date_in)
print('Variable = %s'%var_in)
print('Pressure level = %i hPa'%plev_in)
print('Shape of extracted data matrix is',data.shape)

### Plot data

In [ ]:
####################################
## Plotting parameters
lon_min_pl = 0
lon_max_pl = 360
lat_min_pl = -90
lat_max_pl = 90
var_in_units = 'K'  #if dataset includes this then you should read it in automatically

##Define colors, colorbar
data_min_pl = 250
data_max_pl = 330
data_delt = 5
# data_min_pl = np.nanmin(data)
# data_max_pl = np.nanmax(data)
# data_delt = (data_max_pl-data_min_pl)/20
###################################

# set up the figure
plt.figure()
fig=plt.figure(figsize=(12, 8) )

# Plot the field using Basemap.  Start with setting the map
# projection using the limits of the lat/lon data itself:

# Miller projection:
m=Basemap(projection='mill',lat_ts=10,llcrnrlon=lon_min_pl, \
  urcrnrlon=lon_max_pl,llcrnrlat=lat_min_pl,urcrnrlat=lat_max_pl, \
  resolution='c')

## If desired, regrid data to regular coarse grid (e.g. for non-gridded data)
# dlon = 1.0  #[deg]
# dlat = dlon #[deg]
# lons_new=np.arange((180+dlon)*(2/dlon))*dlon-180.0
# lats_new=np.arange((90+dlat)*(2/dlat))*dlat-90.0
# #print(lons_new,lats_new)
# from matplotlib.mlab import griddata
# data_new = griddata(lon, lat, data, lons_new, lats_new, interp='linear')
# print(lons_new.shape,lats_new.shape,data_new.shape)
# lon = lons_new
# lat = lats_new
# data = data_new

# convert the lat/lon values to x/y projections for a Cartesian plot
x, y = m(*np.meshgrid(lon,lat))

## FIRST: TERRAIN (if you have it)
# delev = 200  #[m]
# elev_min = 400  #[m]
# elev_max = 5000  #[m]
# contour_vals = np.arange(elev_min,elev_max,delev)
# CS = m.contour(x, y, zsfc_new, contour_vals,
#                  colors='grey', linewidths=0.4  # negative contours will be dashed by default
#                  )
##Terrain contour labels (if desired)
# clevs = CS.levels[1::3]  #every 3rd level
# clevs = np.arange(1000,np.max(CS.levels),1000)
# plt.clabel(CS, clevs, inline=1, fontsize=10, fmt='%4i')

## SECOND: DATA
# plot the field using the fast pcolormesh routine 

# define the colormap -- discrete colorbar (https://stackoverflow.com/questions/14777066/matplotlib-discrete-colorbar)
cmap = plt.cm.jet
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey (if desired)
#             cmaplist[0] = (.5,.5,.5,1.0)
# create the new map
cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
N_colors = np.int16(((data_max_pl-data_min_pl)/data_delt)+1)
bounds = np.linspace(data_min_pl,data_max_pl,N_colors)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

m.pcolormesh(x,y,data,shading='flat',cmap=cmap, norm=norm)
m.colorbar(location='right')

# Add a coastline and axis values.

m.drawcoastlines()
#m.fillcontinents()
m.drawmapboundary()
m.drawparallels(np.arange(-90.,90.,30.),labels=[1,0,0,0])
m.drawmeridians(np.arange(-180.,180.,30.),labels=[0,0,0,1])

plt.title('%s [%s] %s %i hPa, ERA-I, direct from NCAR RDA (no downloading!)'%(var_in,var_in_units,date_in,plev_in))